# Mesh generation for tutorial 07

In [ ]:
import numpy as np
import pygmsh
from mpi4py import MPI
from dolfinx import MeshTags
from dolfinx.io import ufl_mesh_from_gmsh, XDMFFile
from dolfinx.mesh import create_mesh, locate_entities
from dolfinx.plotting import plot

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters

In [ ]:
r = 1
lcar = 2

### Create mesh

Generate a simple mesh consisting in an hexagon discretized with six equilateral triangle cells.

In [ ]:
geom = pygmsh.geo.Geometry()
c0 = geom.add_circle([0.0, 0.0, 0.0], r, lcar, num_sections=6)
pygmsh_mesh = pygmsh.generate_mesh(geom)

In [ ]:
cells, x = pygmsh_mesh.cells_dict["triangle"], pygmsh_mesh.points[:, :2]
mesh = create_mesh(MPI.COMM_WORLD, cells, x, ufl_mesh_from_gmsh("triangle", x.shape[1]))

In [ ]:
plot(mesh)

### Create cell restrictions

Define mesh tags on cells, which are equal to one on all cells.

In [ ]:
cell_entities_all = locate_entities(
    mesh, mesh.topology.dim, lambda x: np.full((x.shape[1], ), True))
cell_values_all = np.full(cell_entities_all.shape, 1, dtype=np.intc)
cell_restriction_all = MeshTags(mesh, mesh.topology.dim, cell_entities_all, cell_values_all)
cell_restriction_all.name = "cell_restriction_all"

Define mesh tags on cells, which are equal to one on one half of the cells

In [ ]:
eps = np.finfo(float).eps
cell_entities_subset = locate_entities(
    mesh, mesh.topology.dim,
    lambda x: np.logical_or(x[0] < eps, np.logical_and(x[1] < eps, x[0] < 0.5 + eps)))
cell_values_subset = np.full(cell_entities_subset.shape, 1, dtype=np.intc)
cell_restriction_subset = MeshTags(mesh, mesh.topology.dim, cell_entities_subset, cell_values_subset)
cell_restriction_subset.name = "cell_restriction_subset"

### Create facet restrictions

Define mesh tags on facets, which are equal to one on all facets

In [ ]:
facet_entities_all = locate_entities(
    mesh, mesh.topology.dim - 1, lambda x: np.full((x.shape[1], ), True))
facet_values_all = np.full(facet_entities_all.shape, 1, dtype=np.intc)
facet_restriction_all = MeshTags(mesh, mesh.topology.dim - 1, facet_entities_all, facet_values_all)
facet_restriction_all.name = "facet_restriction_all"

Define mesh tags on facets, which are equal to one on two facets

In [ ]:
facet_entities_subset = locate_entities(
    mesh, mesh.topology.dim - 1, lambda x: np.fabs(x[1] + np.sqrt(3) * x[0]) < 0.01)
facet_values_subset = np.full(facet_entities_subset.shape, 1, dtype=np.intc)
facet_restriction_subset = MeshTags(mesh, mesh.topology.dim - 1, facet_entities_subset, facet_values_subset)
facet_restriction_subset.name = "facet_restriction_subset"

### Save

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "hexagon.xdmf", "w") as output:
    output.write_mesh(mesh)
    output.write_meshtags(cell_restriction_all)
    output.write_meshtags(cell_restriction_subset)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    output.write_meshtags(facet_restriction_all)
    output.write_meshtags(facet_restriction_subset)